# Customer Lifetime Value Analysis
### Data Science for Product Managers HW1
### Audrey Zhang

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
from dateutil.relativedelta import *

In [69]:
df = pd.read_csv('HW1 clv_transactions.csv')

##### explore dataset

In [70]:
# check out dataframe 
df.head()

,TransactionID,TransactionDate,CustomerID,Amount
0,1,9/4/2012,1,20.26
1,2,5/15/2012,2,10.87
2,3,5/23/2014,2,2.21
3,4,10/24/2014,2,10.48
4,5,10/13/2012,2,3.94


There are no null values in the dataset:

In [71]:
df.info()
# there are no null (missing) values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4186 entries, 0 to 4185
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    4186 non-null   int64  
 1   TransactionDate  4186 non-null   object 
 2   CustomerID       4186 non-null   int64  
 3   Amount           4186 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 130.9+ KB


In [72]:
# cast date values to datetime format
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

Next, check out the date ranges. The earliest transaction is 2010/01/04 and the latest is 2015/12/31.

In [73]:
df['TransactionDate'].min()

Timestamp('2010-01-04 00:00:00')

In [74]:
df['TransactionDate'].max()

Timestamp('2015-12-31 00:00:00')

There are 1000 unique customers in this dataset.

In [75]:
len(df['CustomerID'].unique())

1000

Next, take a look at the data distribution of the 'amount' field. The minimum value is -1 and the maximum is 2,033. The standard deviation is 38.72. The variance is 1499.54. 

In [76]:
df['Amount'].describe()

count    4186.000000
mean        9.028082
std        38.723909
min        -1.000000
25%         3.310000
50%         6.885000
75%        11.690000
max      2033.000000
Name: Amount, dtype: float64

In [77]:
df['Amount'].var()

1499.5411107517461

It is unclear why there are observations with -1 in the Amount column. This warrants further investigation.

In [78]:
df[df['Amount']<0]

,TransactionID,TransactionDate,CustomerID,Amount
49,50,2012-05-12,13,-1.0
3326,3327,2014-01-16,811,-1.0


It looks like there are only two such transactions, which may be outliers. They may warrant being removed from the analysis.

In [79]:
import plotly.express as px
fig = px.box(df, y="Amount")
fig.show()

There are clearly several outliers in the data. Most of the observations lie between 0 and 40 dollars, with the median at $6.8 dollars. It is extremely likely that the high outliers with amount values within the 1000-2000 range are outliers due to data entry errors (forgetting the decimal separator). Since there are only a few such cases, they will be removed from the dataset from analysis.

In [80]:
df[df['Amount']>50]

,TransactionID,TransactionDate,CustomerID,Amount
23,24,2011-10-07,7,1003.0
211,212,2014-05-14,54,1027.0
4177,4178,2012-08-13,1000,2033.0


In [81]:
# remove outliers with negative amount values 
df = df[(df['Amount']<50) & (df['Amount']>0)].copy()

5 total rows with outlier values are removed.

In [82]:
df.shape

(4181, 4)

To confirm that the rest of the data are good data, plot a histogram of the amount column. Clearly, this is a skewed distribution with a right tail.

In [83]:
fig = px.histogram(df, x="Amount")
fig.show()

##### Customer Cohort Assignment

First, create variable for the year of acquisition. This is based on the earliest transaction date that could be found for each customer. 

In [84]:
# first get the earliest transaction date per customer 
acquisition_df = df.sort_values('TransactionDate').groupby('CustomerID').min().reset_index()

In [194]:
# next assign eac customer with his/her acquisition year 
acquisition_df['cohort'] = acquisition_df['TransactionDate'].dt.year.apply(lambda x: datetime.datetime(x,1, 1)).dt.date

In [195]:
acquisition_df

,CustomerID,TransactionID,TransactionDate,Amount,cohort
0,1,1,2012-09-04,20.26,2012-01-01
1,2,2,2012-05-15,1.59,2012-01-01
2,3,10,2012-11-26,1.67,2012-01-01
3,4,17,2015-07-07,0.54,2015-01-01
4,5,18,2015-01-24,6.90,2015-01-01
...,...,...,...,...,...
995,996,4152,2012-06-27,5.06,2012-01-01
996,997,4154,2014-10-14,7.93,2014-01-01
997,998,4156,2010-09-29,5.46,2010-01-01
998,999,4164,2011-04-18,0.80,2011-01-01


In [197]:
# merge the cohort variable back into the original dataset
df = pd.merge(df, acquisition_df[['CustomerID', 'cohort']], how='left', on='CustomerID')

In [199]:
# spot check 
df[df['CustomerID']==2]

,TransactionID,TransactionDate,CustomerID,Amount,cohort
1,2,2012-05-15,2,10.87,2012-01-01
2,3,2014-05-23,2,2.21,2012-01-01
3,4,2014-10-24,2,10.48,2012-01-01
4,5,2012-10-13,2,3.94,2012-01-01
5,6,2013-01-23,2,12.37,2012-01-01
6,7,2014-03-09,2,1.59,2012-01-01
7,8,2013-11-17,2,2.75,2012-01-01
8,9,2014-10-10,2,3.51,2012-01-01


##### Calculate cumulative transactions

For cumulative transactions within a given age (e.g. 12 months), we need to find the transactions that occured within 12 months of the first date of the year of acquisition (e.g. 2010-01-01 - 2010-12-31). The first step is to calculate the difference in months between the transaction date and the cohort date.

In [201]:
df['timediff'] = ((df['TransactionDate'].dt.date - df['cohort'])/np.timedelta64(1, 'M')).apply(lambda x: math.ceil(x))

In [202]:
df.head()

,TransactionID,TransactionDate,CustomerID,Amount,cohort,timediff
0,1,2012-09-04,1,20.26,2012-01-01,9
1,2,2012-05-15,2,10.87,2012-01-01,5
2,3,2014-05-23,2,2.21,2012-01-01,29
3,4,2014-10-24,2,10.48,2012-01-01,34
4,5,2012-10-13,2,3.94,2012-01-01,10


In [281]:
time_periods = [12, 24, 36, 48, 60, 72] 
cohorts = sorted(list(df['cohort'].unique()))
transactions = [] 
latest_transaction = df['TransactionDate'].max()

for c in cohorts:
    tr = []
    for t in time_periods:
        if c + relativedelta(months=+t-12) < latest_transaction:
            cumulative = df[(df['cohort']==c) & (df['timediff'] <= t)]['Amount'].sum().round(2)
            tr.append(cumulative)
        else:
            tr.append(None)
    transactions.append(tr)    

In [282]:
df['Origin'] = df['cohort'].apply(lambda x: 
                                  (x.strftime('%Y-%m-%d') + 
                                  ' - ' + (x + relativedelta(days=364)).strftime('%Y-%m-%d')))

In [283]:
origin_list = sorted(list(df['Origin'].unique()))

In [284]:
cumulative_transactions = pd.DataFrame(transactions, columns = time_periods, index = origin_list)

In [304]:
cumulative_transactions.fillna('')

,12,24,36,48,60,72
2010-01-01-2010-12-31,2262.13,3614.78,5274.81,6632.37,7930.69,8964.49
2011-01-01-2011-12-31,2252.70,3757.9,5465.99,6703.11,7862.24,
2012-01-01-2012-12-30,2181.35,3874.69,5226.86,6501.85,,
2013-01-01-2013-12-31,2179.85,3609.81,5227.75,,,
2014-01-01-2014-12-31,1830.85,3262.05,,,,
2015-01-01-2015-12-31,1912.17,,,,,


##### Calculate new customer count

In [299]:
customers = []

for c in cohorts:
    cs = []
    for t in time_periods:
        if c + relativedelta(months=+t-12) < latest_transaction:
            ct = len(df[df['cohort'] == c]['CustomerID'].unique())
            cs.append(ct)
        else:
            cs.append(None)
    customers.append(cs)  

In [300]:
customer_counts = pd.DataFrame(customers, columns = time_periods, index = origin_list)

In [302]:
customer_counts.fillna('')

,12,24,36,48,60,72
2010-01-01-2010-12-31,172,172,172,172,172,172
2011-01-01-2011-12-31,170,170,170,170,170,
2012-01-01-2012-12-30,163,163,163,163,,
2013-01-01-2013-12-31,180,180,180,,,
2014-01-01-2014-12-31,155,155,,,,
2015-01-01-2015-12-31,160,,,,,


##### Calculate historic CLV

In [327]:
historic_clv = cumulative_transactions.div(customer_counts).round(2)

In [328]:
historic_clv.fillna('')

,12,24,36,48,60,72
2010-01-01-2010-12-31,13.15,21.02,30.67,38.56,46.11,52.12
2011-01-01-2011-12-31,13.25,22.11,32.15,39.43,46.25,
2012-01-01-2012-12-30,13.38,23.77,32.07,39.89,,
2013-01-01-2013-12-31,12.11,20.05,29.04,,,
2014-01-01-2014-12-31,11.81,21.05,,,,
2015-01-01-2015-12-31,11.95,,,,,


In [340]:
fig = px.scatter(historic_clv.transpose(), labels={
     "value": "Historic CLV ($)",
     "index": "Age (months)",
     "variable": "Origin"
     }).update_traces(mode='lines+markers')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = time_periods
    ),
    title = 'Historic CLV'
)

fig.show()

Next, calculate the single combined curve of Historic CLV by taking the weighted average of the historic CLV at each Age. 

In [361]:
combined_clv =pd.DataFrame((historic_clv * customer_counts).sum().div(customer_counts.sum()).round(2), columns = ['HistoricCLV']).reset_index().rename(columns = {'index' : 'Age'})

In [362]:
combined_clv

,Age,HistoricCLV
0,12,12.62
1,24,21.57
2,36,30.94
3,48,39.28
4,60,46.18
5,72,52.12


##### Interpretation 

Customers acquired in 2011 have spent $7,862.24 to date.

In [368]:
cumulative_transactions.loc['2011-01-01-2011-12-31'].max()

7862.24

Based on the visualization above, each cohort (ie. origin) group typically increases their cumulative spending (and therefore their historic CLV) in a similar upward trajectory (i.e. the slopes of the lines are relatively similar). However, when comparing the historic CLV between groups at different age cutoffs, it is clear that customers acquired in 2013 generally have higher CLV than customers acquired other years. This is a cohort of high value customers and retention efforts should focus on this group, as they represent higher CLV for the business. 

Customers acquired in 2013 seems to have lower historic CLV compared to other cohorts. This is alarming for the business, and indicate that additional marketing efforts should be targeted towards this cohort in order to increase their CLV. Additionally, at the 12 month mark, customers in the later cohorts (2013-2015) have lower CLV compared to earlier customer cohorts, suggesting that additional marketing at the onset of customer acquistion should be incorporated in order to increase the amount of cumulative transactions per customers during the first 12 months of their customer history. 